In [1]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
import time
import pydde as d

In [35]:
#Parameters
samplenum = 4
input_size = 3
output_size = 3
time_length = 3; #ms

In [37]:
# Generate simulation
dyn = d.PyDyn('../Data/point-mass_pendulum.sim', time_length)
state_init = dyn.compute(dyn.p_init)
f = dyn.f(state_init, dyn.p_init)
df = dyn.df_dp(state_init, dyn.p_init)
dy = dyn.dy_dp(state_init, dyn.p_init)
#Sample targets only variables in z direction
y_target = np.zeros((samplenum, 3))
y_target[:,2] = np.random.rand(samplenum)
y_target[:,1] = 2
#print(y_target)

###################################
#USE P FROM TRAJ_OPT
p = np.ones((3*time_length, samplenum))
for i in range(samplenum):
    p[:,i] = dyn.get_p(y_target.transpose()[:,i], dyn.p_init)
p = torch.tensor(p, requires_grad = True).t()#Sample targets only variables in z direction
y_target = np.zeros((samplenum, 3))
y_target[:,2] = np.random.rand(samplenum)
y_target[:,1] = 2
#print(y_target)

###################################
#USE P FROM TRAJ_OPT
p = np.ones((3*time_length, samplenum))
for i in range(samplenum):
    p[:,i] = dyn.get_p(y_target.transpose()[:,i], dyn.p_init)
p = torch.tensor(p, requires_grad = True).t()
p.shape

torch.Size([4, 9])

In [30]:
##############################
#USE P_INIT ONLY
p = np.ones((3*time_length, samplenum))
for i in range(samplenum):
    p[:,i] = dyn.p_init
p = torch.tensor(p, requires_grad = True).t()
input = p.double()
print(p)
print(p.shape)

tensor([[0., 3., 0., 0., 3., 0., 0., 3., 0.],
        [0., 3., 0., 0., 3., 0., 0., 3., 0.],
        [0., 3., 0., 0., 3., 0., 0., 3., 0.],
        [0., 3., 0., 0., 3., 0., 0., 3., 0.]], dtype=torch.float64,
       grad_fn=<TBackward>)
torch.Size([4, 9])


## Building the custon Simulation Activation Function

In [33]:
#MODEL FOR TESTS

class Simulate(torch.autograd.Function):
    
    @staticmethod
    def forward(ctx, input):
        #print(f'input: {input}')
        p = input.clone().numpy().transpose()
        y_pred = torch.ones([samplenum,3])
        for i in range(len(p[0, :])):
            state = dyn.compute(p[:,i])
            y_pred[i, :] = torch.tensor(state.y[-3:])
        print(f'y_pred: {y_pred}')
        
        ctx.save_for_backward(input)
        
        return y_pred
        
    @staticmethod
    def backward(ctx, grad_output):
        input, = ctx.saved_tensors
        p = input.clone().numpy().transpose()
        dy_dp_batch = torch.zeros([3, 3*time_length])
        for i in range(samplenum):
            state= dyn.compute(p[:, i])
            dy_dp = dyn.dy_dp(state, p[:, i])
            dy_dp = torch.tensor(dy_dp[-3:, :])
            dy_dp_batch = dy_dp_batch + dy_dp
        print(f'shape of dy/dp_batch: {dy_dp_batch.shape}')
        
        grad_input = torch.tensor(grad_output.double().mm(dy_dp_batch))/samplenum
        print(f'shape of grad input: {grad_input.shape}')
        print(f'shape of grad output: {grad_output.shape}')
        print(f'grad_input: {grad_input}')
        print(f'grad_output: {grad_output}')
        return grad_input    

Simulate = Simulate.apply

In [38]:
################################
#MODEL

class Simulate(torch.autograd.Function):
    
    @staticmethod
    def forward(ctx, input):
        #print(f'input: {input.shape}')
        p = input.clone().numpy().transpose()
        y_pred = torch.ones([len(p[0, :]),3])
        for i in range(len(p[0, :])):
            state = dyn.compute(p[:,i])
            y_pred[i, :] = torch.tensor(state.y[-3:])
        #print(f'y_pred: {y_pred.shape}')
        
        ctx.save_for_backward(input)
        
        return y_pred
        
    @staticmethod
    def backward(ctx, grad_output):
        #print(grad_output)
        input, = ctx.saved_tensors
        p = input.clone().numpy().transpose()
        dy_dp_batch = torch.zeros([3, 3*time_length])
        for i in range(len(p[0, :])):
            state= dyn.compute(p[:, i])
            dy_dp = dyn.dy_dp(state, p[:, i])
            dy_dp = torch.tensor(dy_dp[-3:, :])
            dy_dp_batch = dy_dp_batch + dy_dp
        #print(f'dy/dp_batch: {dy_dp_batch/samplenum}')
        
        grad_input = torch.tensor(grad_output.double().mm(dy_dp_batch/samplenum))
        #print(f'shape of grad input: {grad_input.shape}')
        #print(f'shape of grad output: {grad_output.shape}')
        return grad_input

Simulate = Simulate.apply

In [42]:
#GRADCHECK
from torch.autograd import gradcheck

test = gradcheck(Simulate, (input,), eps=1e-3, atol=1e-4, raise_exception = True)
print(test)

True
